In [120]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
import pandas as pd
import xgboost
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

In [134]:
df = pd.read_excel("cleaned_data.xlsx")
df = df.drop(['Unnamed: 0','other_details','images'],axis = 1)
df.head()

,price,accepted_currency,city,sub,no_of_bedrooms,no_of_bathrooms
0,125000.0,GHC,Accra,Other,4,4
1,2822500.0,USD,Accra,Legon,4,5
2,10725500.0,USD,Accra,Cantonments,5,6
3,2929755.0,USD,Accra,Airport,2,2
4,2032200.0,USD,Accra,Other,2,2


In [135]:
# vectorizer = CountVectorizer(stop_words='english')
# X_text = vectorizer.fit_transform(df['other_details'])

# Combine BoW features with tabular data
# StandardScaler().fit_transform(X_tabular)
# X_combined = np.hstack ((X_tabular, X_text.toarray()))

In [136]:
X_text.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]], dtype=int64)

In [170]:
ct = ColumnTransformer([
                   ("Categorical",OrdinalEncoder(),['city','sub','accepted_currency']),
                   #("Numerical",StandardScaler(),['no_of_bedrooms','no_of_bathrooms']),
#                    ("Upsampler",SMOTE(sampling_strategy=0.1),list(df.columns))
                    
                      ],remainder = 'passthrough')



In [172]:
pipeline = Pipeline([("transformer",ct),("model",AdaBoostRegressor())])

In [173]:
pipeline.get_params

<bound method Pipeline.get_params of Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Categorical',
                                                  OrdinalEncoder(),
                                                  ['city', 'sub',
                                                   'accepted_currency'])])),
                ('model', AdaBoostRegressor())])>

In [174]:
params = {
    "model":[RandomForestRegressor(),AdaBoostRegressor(),LinearRegression(),DecisionTreeRegressor()]
}

In [175]:
gscv = GridSearchCV(estimator=pipeline,
                    param_grid=params,cv = 5,
                    error_score = 'raise')

In [155]:
X = df.iloc[:,df.columns != 'price']
y = df['price']
display(y)

0        125000.0
1       2822500.0
2      10725500.0
3       2929755.0
4       2032200.0
          ...    
392     1278400.0
393     2857600.0
394     1504000.0
395      550000.0
396     2030400.0
Name: price, Length: 397, dtype: float64

In [156]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=11)

In [158]:
#SMOTE(sampling_strategy=0.1).fit(X_train,y_train)

In [176]:
gscv.fit(X_train,y_train)

GridSearchCV(cv=5, error_score='raise',
             estimator=Pipeline(steps=[('transformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('Categorical',
                                                                         OrdinalEncoder(),
                                                                         ['city',
                                                                          'sub',
                                                                          'accepted_currency'])])),
                                       ('model', AdaBoostRegressor())]),
             param_grid={'model': [RandomForestRegressor(), AdaBoostRegressor(),
                                   LinearRegression(),
                                   DecisionTreeRegressor()]})

In [177]:
gscv.best_score_

0.17810827266408655